In [1]:
import pandas as pd
import networkx as nx
from itertools import combinations

# Load the CSV file
df = pd.read_csv("participants.csv")


In [12]:
df.columns

Index(['First name', 'Last name', 'Email address',
       'Would you prefer to work on the project in a team?',
       'Preferred Teammates', 'Full Name'],
      dtype='object')

In [5]:
# Example CSV Columns: 'First Name', 'Last Name', 'Preferred Teammates'
df['Full Name'] = df['First name'] + " " + df['Last name']

In [11]:
df = df.rename(columns={"Are there specific individuals you would prefer to work with? If yes, please list their names or emails below." : "Preferred Teammates"})

In [13]:
# Create a graph to represent preferences
G = nx.Graph()

# Add participants as nodes
for participant in df['Full Name']:
    G.add_node(participant)

# Add edges for preferences
for _, row in df.iterrows():
    person = row['Full Name']
    if pd.notna(row['Preferred Teammates']):
        preferred = row['Preferred Teammates'].split(", ")
        for teammate in preferred:
            if teammate in df['Full Name'].values:
                G.add_edge(person, teammate)

# Find groups using connected components and fill gaps
groups = []
used = set()
for component in nx.connected_components(G):
    component = list(component)
    while len(component) >= 6:
        groups.append(component[:6])
        used.update(component[:6])
        component = component[6:]
    if component:
        groups.append(component)  # Add incomplete group

# Fill remaining slots with unassigned participants
remaining = [name for name in df['Full Name'] if name not in used]
for i, participant in enumerate(remaining):
    if i % 6 == 0:
        groups.append([])
    groups[-1].append(participant)

# Save the groups to a CSV file
group_df = pd.DataFrame({
    f"Group {i + 1}": group for i, group in enumerate(groups)
})
group_df.to_csv("group_assignments.csv", index=False)

ValueError: All arrays must be of the same length